# Swept Langmuir Prototype: Sweep Finder

This notebook will focus on the mechanics of finding the voltage sweep intervals in recorded signals.  **This notebook is for prototyping purposes and will either be deleted or converted into a documentaiton notebook when the protoyping is finished.**

### Table of Contents

1. [Import Packages](#Import-Packages)
1. [Enable the Swept Langmuir Diagnostic](#Enable-the-Swept-Langmuir-Diagnostic)
1. [Build the `xarray.Dataset`](#Build-the-xarray.Dataset)

## Import Packages

In [ ]:
%matplotlib inline

import astropy.units as u
import h5py
import numpy as np
import os
import pandas as pd
import sys
import xarray as xr

from bapsflib import lapd
from matplotlib import pyplot as plt
from numbers import Number
from scipy import signal
from typing import Mapping, Hashable, Any
from warnings import warn
from xarray.core.utils import either_dict_or_kwargs
from xarray.core.coordinates import remap_label_indexers

from plasmapy import __file__ as plasmapy_file
from plasmapy.diagnostics import xdiagnostics
from plasmapy.utils.exceptions import PlasmaPyError, PlasmaPyWarning

plasmapy_dir = os.path.dirname(os.path.abspath(plasmapy_file))

In [ ]:
def smooth(x, window_len=11, window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s = np.r_[x[window_len-1:0:-1], x, x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = eval('np.' + window + '(window_len)')

    y = np.convolve(w/w.sum(), s, mode='valid')
    return y

## Enable the Swept Langmuir Diagnostic

By default `plasmapy` does not register any of its `xarray.Dataset` accessors to avoid polution of the namespace and reduce overhead of importing unnecessary code.  Instead, we provide the `xdiagnostics` object so the user can enable the `xarray.Dataset` accessor(s) he or she wants.  Just executing `xdiagnostics` will show what diagnosics are available and enabled/disabled.

In [ ]:
xdiagnostics.enable('swept_langmuir', rename="slang")
xdiagnostics

## Build the `xarray.Dataset`

- The `Dataset` needs to have two `DataArray`s named `voltage` and `current`.
- The `DataArray`'s are either 1D or 2D.
- The `DataArray`'s have to have matching `dims` and `shape`.
- The names of the dimesions do not matter, since the **diagnostic** is agnostic towards the names but the order does matter.
  - Dims for 1D: `dims=["time_index"]`
  - Dims for 2D: `dims=["signal_index", "time_index"]`
  
The HDF5 dataset being used is from the file ``/data/BAPSF_Data/ICRF_campaign/October2019_MacorCopperHP/14a_Lang_Plane1_12kV 2019-10-31 19.33.40.hdf5`` located on midas.  Voltage is on board 2, channel 6 and Current ins on board 2, channel 7.

In [ ]:
# Define file name an location

fdir = os.path.abspath(os.path.join(plasmapy_dir, "../", "docs", "notebooks", "langmuir_samples"))
fname = "14a_Lang_Plane1_12kV 2019-10-31 19.33.40.hdf5"

fdir = os.path.join(os.path.expanduser("~/"), "Documents", "Research", "BaPSF", "Projects", "msi-langmuir", "SmPD_tests", "20190410_swept_lang")
fname = "03-msilang-swept-4000utorr_2019-04-10_18.07.14.hdf5"

fpath = os.path.join(fdir, fname)

# Open HDF5 and get data
with lapd.File(fpath, silent=True) as f:
#         vdata = f.read_data(2, 6, index=slice(5))
#         idata = f.read_data(2, 7, index=slice(5))
    vdata = f.read_data(2, 1, index=slice(5), silent=True)
    vdata['signal'] = 50.0 * vdata['signal']
    idata = f.read_data(2, 2, index=slice(5), silent=True)
    idata['signal'] = 20.0 * idata['signal']
    
    f.run_description()

In [ ]:
# Build XArrays
vda = xr.DataArray(vdata['signal'], dims=["shotnum", "time"], name="voltage")
ida = xr.DataArray(idata['signal'], dims=["shotnum", "time"], name="current")

ds = xr.Dataset(
    {vda.name: vda,
     ida.name: ida},
    coords={
        "shotnum": ("shotnum", vdata["shotnum"]),
        "x": ("shotnum", vdata['xyz'][..., 0]),
        "y": ("shotnum", vdata['xyz'][..., 1]),
        "z": ("shotnum", vdata['xyz'][..., 2]),
    },
)
ds.slang
ds

In [ ]:
isig = 0

volt = ds.voltage[isig, ...]
svolt = smooth(volt, window_len=swin_len)
diff_svolt = np.diff(svolt)

# swin_len = 300
swin_len = np.max([int(np.round(volt.size * 0.002)), 3])
print(swin_len)

print(f"volt size = {volt.size}")
print(f"smoothed size = {svolt.size}")
print(f"dif smoothed size = {diff_svolt.size}")

diff_peaks = signal.find_peaks(np.abs(diff_svolt), prominence=(None, None), width=(None, None), rel_height=0.9)
norm_peaks = signal.find_peaks(svolt, prominence=(None, None), width=(None, None), rel_height=0.99)

prominences = diff_peaks[1]['prominences']
mask_diff_peaks = np.where(prominences / prominences.max() > 0.9)[0]

prominences = norm_peaks[1]['prominences']
mask_norm_peaks = np.where(prominences / prominences.max() > 0.9)[0]

In [ ]:
# mask_all_peaks
# mip = mask_all_peaks[0]
# mip
# all_peaks[1]["right_bases"][mip]

In [ ]:
width = 20.
fig = plt.figure(figsize=(width, 3. * (9./28.) * width))

volt = ds.voltage[0,...]
ipeaks = diff_peaks[0][mask_diff_peaks]

# # top plot
ax0 = fig.add_subplot(311)
volt.plot()
ax0.plot(svolt)
ylim = ax0.get_ylim()
for mip in mask_diff_peaks:
    ip = diff_peaks[0][mip]
    lip = diff_peaks[1]['left_ips'][mip] - (0.25 * swin_len)
    rip = diff_peaks[1]['right_ips'][mip] + (0.25 * swin_len)
    ax0.plot([ip, ip], ylim, 'r', linewidth=3)
    
    xfill = [lip, rip, rip, lip]
    yfill = [ylim[0], ylim[0], ylim[1], ylim[1]]
    plt.fill(xfill, yfill, "r", alpha=0.3)
for ii, mip in enumerate(mask_norm_peaks):
    if ii != 4:
        continue
    ip = norm_peaks[0][mip]
    lip = norm_peaks[1]['left_ips'][mip]  # - (0.4 * swin_len)
    rip = norm_peaks[1]['right_ips'][mip]  # + (0.4 * swin_len)
    ax0.plot([ip, ip], ylim, 'c', linewidth=3)
    
    xfill = [lip, rip, rip, lip]
    yfill = [ylim[0], ylim[0], ylim[1], ylim[1]]
    plt.fill(xfill, yfill, "c", alpha=0.3)

# 2nd plot
ax1 = fig.add_subplot(312)
ax1.plot(np.abs(diff_svolt))
ax1.plot(ipeaks, np.abs(diff_svolt)[ipeaks], 'ro')

# 3rd plot
ax2 = fig.add_subplot(313)
ii = 4

mip = mask_diff_peaks[ii]
ip = diff_peaks[0][mip]
lip = diff_peaks[1]['left_ips'][mip] - (0.25 * swin_len)
rip = diff_peaks[1]['right_ips'][mip] + (0.25 * swin_len)
plt.plot([ip, ip], ylim, 'r', linewidth=3)
xfill = [lip, rip, rip, lip]
yfill = [ylim[0], ylim[0], ylim[1], ylim[1]]
plt.fill(xfill, yfill, "r", alpha=0.3)

wl = lip
wr = rip

mip = mask_norm_peaks[ii]
ip = norm_peaks[0][mip]
lip = norm_peaks[1]['left_ips'][mip]  # - (0.4 * swin_len)
rip = norm_peaks[1]['right_ips'][mip]  # + (0.4 * swin_len)
plt.plot([ip, ip], ylim, 'c', linewidth=3)

xfill = [lip, rip, rip, lip]
yfill = [ylim[0], ylim[0], ylim[1], ylim[1]]
plt.fill(xfill, yfill, "c", alpha=0.3)

wl = int(np.min([lip, wl])) - swin_len
wr = int(np.max([rip, wr])) + swin_len
plt.plot(np.arange(wl, wr), volt[wl:wr])
plt.plot(np.arange(wl, wr), svolt[wl:wr])

In [ ]:
all_peaks = signal.find_peaks(svolt, prominence=(None, None), width=(None, None), rel_height=0.8)
prominences = all_peaks[1]['prominences']
mask_all_peaks = np.where(prominences / prominences.max() > 0.9)[0]

for mip in mask_all_peaks:
    ip = all_peaks[0][mip]
    lip = all_peaks[1]['left_ips'][mip]  # - (0.4 * swin_len)
    rip = all_peaks[1]['right_ips'][mip]  # + (0.4 * swin_len)
    width = all_peaks[1]['widths'][mip]
    
    print(f"{lip - 0.05*width} -> {ip} -> {rip + 0.05*width} .. Delta = {width}")

# plt.plot(all_peaks[0], prominences/prominences.max(), 'bo')
# plt.plot(svolt)